# Day 15

## Part 1

- Today's challenge details a hashing algorithm:
    - start with value = 0
    - value += ascii(char)
    - value *= 17
    - value %= 256
    - repeat for each char
- Find the sum of the hashed initialisation sequence values

In [1]:
from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig, markdown
)

parser = ParseConfig(',', str)

codes = parse_from_file('puzzle_input\\day_15.txt', parser)

In [3]:
def get_hash(code: str) -> int:
    """returns a byte value of the code string"""
    value = 0
    for char in code:
        value += ord(char)
        value *= 17
        value %= 256
    return value

example = [
    'rn=1','cm-','qp=3','cm=2','qp-','pc=4','ot=9','ab=5','pc-','pc=6','ot=7']

for code in example:
    print(code, get_hash(code))

rn=1 30
cm- 253
qp=3 97
cm=2 47
qp- 14
pc=4 180
ot=9 9
ab=5 197
pc- 48
pc=6 214
ot=7 231


In [4]:
# cool that works let's solve!
hash_sum = sum([get_hash(code) for code in codes])

markdown(
    '### Solution',
    f'The total hash sum is: {hash_sum}'
)

### Solution
The total hash sum is: 515210

## Part 2

- the sequence tells you what boxes to put lenses into
- The box number is the hash of the command
- the lens label is the first letters of the codes
- a `-` means take out the lens with that label from the box
- an `=` has a focal length afterwards (from 1 to 9)
    - replace a lens with the same label in that box if there is one
    - otherwise put it at the back of lenses in the box
- the total focusing power is the solution. This is found per lens by finding the product of:
    - 1 + box number
    - Slot number of the lense in the box (starting at 1)
    - the focal length of the lens

In [13]:
from dataclasses import dataclass

from tqdm import tqdm

In [12]:
@dataclass
class Lens:
    label: str
    focal_length: int

@dataclass
class Box:
    number: int
    lenses: list[Lens]

    def insert_lens(self, lens: Lens) -> None:
        """inserts a lens into this box in the right place"""
        for index, item in enumerate(self.lenses):
            if item.label == lens.label:
                self.lenses[index] = lens
                break
        else:
            self.lenses.append(lens)
    
    def remove_lens(self, lens: Lens) -> None:
        """removes a lens from this box"""
        for index, item in enumerate(self.lenses):
            if item.label == lens.label:
                self.lenses.pop(index)
                break
    
    @property
    def focusing_power(self) -> int:
        """calculate the focusing power in the box"""
        power = 0
        for slot, lens in enumerate(self.lenses, start=1):
            power += (1 + self.number) * (slot) * (lens.focal_length)
        return power


In [20]:
# ok let's run the codes!
boxes = [Box(value, []) for value in range(256)]

for code in tqdm(codes):
    # see if it's an insert code
    if '=' in code:
        label, focal = code.split('=')
        box = get_hash(label)
        boxes[box].insert_lens(Lens(label, int(focal)))
    else:
        label = code[:-1]
        box = get_hash(label)
        boxes[box].remove_lens(Lens(code[:-1], None))

total = sum([box.focusing_power for box in boxes])

100%|██████████| 4000/4000 [00:00<00:00, 269051.04it/s]


In [21]:
markdown(
    '### Solution',
    f'The total focusing power is: {total}'
)

### Solution
The total focusing power is: 246762